In [4]:
import os
import numpy as np
import matplotlib.pyplot as plt
import glob
import sys

from tqdm import tqdm

def get_inten(ctype, frac, mode, angle):
    
    I = np.zeros((512 * 16, 120))
    
    specs = glob.glob('./runs/nessy/G2V/matthias_noext/' + ctype + '/' + str(frac) + '/' + mode + '/' + str(angle) + '/spec/*')

    for i in tqdm(range(len(specs)), desc = ctype + ' ' + mode + ' ' + str(angle)):
        
        if os.path.getsize(specs[i]) > 0: I[i, :] = np.loadtxt(specs[i])
            
    return I

def read_nessy_spec(angles):
    
    I0l = np.zeros((512 * 16, 5, 120))
    ISl = np.zeros((512 * 16, 5, 120))
    I3l = np.zeros((512 * 16, 5, 120))
    I0n = np.zeros((512 * 16, 5, 120))
    ISn = np.zeros((512 * 16, 5, 120))
    I3n = np.zeros((512 * 16, 5, 120))
    
    for j in range(len(angles)):
        
        I0l[:, j, :] = get_inten('hydro', 32, 'lte',  angles[j])
        ISl[:, j, :] = get_inten('ssd',   32, 'lte',  angles[j])
        I3l[:, j, :] = get_inten('300G',  32, 'lte',  angles[j])
        
        I0n[:, j, :] = get_inten('hydro', 32, 'nlte', angles[j])
        ISn[:, j, :] = get_inten('ssd',   32, 'nlte', angles[j])
        I3n[:, j, :] = get_inten('300G',  32, 'nlte', angles[j])
                
    return I0l, ISl, I3l, I0n, ISn, I3n

#I0l, ISl, I3l, I0n, ISn, I3n = read_nessy_spec([2, 4, 6, 8, 10])

#np.savez('spec_nessy_G2V_matthias_noext_32', I0l = I0l, ISl = ISl, I3l = I3l, I0n = I0n, ISn = ISn, I3n = I3n)

I0l = np.load('spec_nessy_G2V_matthias_noext_32.npz')['I0l']
ISl = np.load('spec_nessy_G2V_matthias_noext_32.npz')['ISl']
I3l = np.load('spec_nessy_G2V_matthias_noext_32.npz')['I3l']
I0n = np.load('spec_nessy_G2V_matthias_noext_32.npz')['I0n']
ISn = np.load('spec_nessy_G2V_matthias_noext_32.npz')['ISn']
I3n = np.load('spec_nessy_G2V_matthias_noext_32.npz')['I3n']

I0l_m = np.zeros((5, 120))
ISl_m = np.zeros((5, 120))
I3l_m = np.zeros((5, 120))
I0n_m = np.zeros((5, 120))
ISn_m = np.zeros((5, 120))
I3n_m = np.zeros((5, 120))

plt_mu = [4, 3, 2, 1, 0]

for i in plt_mu:

    for j in range(120):
        
        nz0l = np.nonzero(I0l[:, i, j])
        nzSl = np.nonzero(ISl[:, i, j])
        nz3l = np.nonzero(I3l[:, i, j])
        nz0n = np.nonzero(I0n[:, i, j])
        nzSn = np.nonzero(ISn[:, i, j])
        nz3n = np.nonzero(I3n[:, i, j])
    
        I0l_m[i, j] = np.mean(I0l[nz0l, i, j])
        ISl_m[i, j] = np.mean(ISl[nzSl, i, j])
        I3l_m[i, j] = np.mean(I3l[nz3l, i, j])
        I0n_m[i, j] = np.mean(I0n[nz0n, i, j])
        ISn_m[i, j] = np.mean(ISn[nzSn, i, j])
        I3n_m[i, j] = np.mean(I3n[nz3n, i, j])

w = np.arange(1805, 3005, 10) / 10

np.savez('spec_nessy_G2V_matthias_noext_32_mean', I0l = I0l_m, ISl = ISl_m, I3l = I3l_m, I0n = I0n_m, ISn = ISn_m, I3n = I3n_m)

In [5]:
I0l_ext = np.load('spec_nessy_G2V_matthias_ext_32_mean.npz')['I0l']
ISl_ext = np.load('spec_nessy_G2V_matthias_ext_32_mean.npz')['ISl']
I3l_ext = np.load('spec_nessy_G2V_matthias_ext_32_mean.npz')['I3l']
I0n_ext = np.load('spec_nessy_G2V_matthias_ext_32_mean.npz')['I0n']
ISn_ext = np.load('spec_nessy_G2V_matthias_ext_32_mean.npz')['ISn']
I3n_ext = np.load('spec_nessy_G2V_matthias_ext_32_mean.npz')['I3n']

I0l_noext = np.load('spec_nessy_G2V_matthias_noext_32_mean.npz')['I0l']
ISl_noext = np.load('spec_nessy_G2V_matthias_noext_32_mean.npz')['ISl']
I3l_noext = np.load('spec_nessy_G2V_matthias_noext_32_mean.npz')['I3l']
I0n_noext = np.load('spec_nessy_G2V_matthias_noext_32_mean.npz')['I0n']
ISn_noext = np.load('spec_nessy_G2V_matthias_noext_32_mean.npz')['ISn']
I3n_noext = np.load('spec_nessy_G2V_matthias_noext_32_mean.npz')['I3n']

In [12]:
import itertools

plt_mu = [4, 3, 2, 1, 0]

mu = [0.2, 0.4, 0.6, 0.8, 1.0]

pdf_names = ''

for i in tqdm(plt_mu):
    
    plt.close('all')

    fig, ax = plt.subplots(nrows = 3, ncols = 2, figsize = (15, 10))
    
    fig.suptitle(r'$\mu$ = ' + str(mu[i]), y = 1.01)

    fig.tight_layout()

    ax[0, 0].plot(w, I0l_ext[i, :] / I0l_noext[i, :], linewidth = 2)
    ax[0, 1].plot(w, I0n_ext[i, :] / I0n_noext[i, :], linewidth = 2)    
    ax[1, 0].plot(w, ISl_ext[i, :] / ISl_noext[i, :], linewidth = 2)
    ax[1, 1].plot(w, ISn_ext[i, :] / ISn_noext[i, :], linewidth = 2)
    ax[2, 0].plot(w, I3l_ext[i, :] / I3l_noext[i, :], linewidth = 2)
    ax[2, 1].plot(w, I3n_ext[i, :] / I3n_noext[i, :], linewidth = 2)
    
    ax[0, 0].axhline(y = 1.0, linestyle = '--', color = 'black')
    ax[0, 1].axhline(y = 1.0, linestyle = '--', color = 'black')
    ax[1, 0].axhline(y = 1.0, linestyle = '--', color = 'black')
    ax[1, 1].axhline(y = 1.0, linestyle = '--', color = 'black')
    ax[2, 0].axhline(y = 1.0, linestyle = '--', color = 'black')
    ax[2, 1].axhline(y = 1.0, linestyle = '--', color = 'black')

    ax[0, 0].set_title('LTE')
    ax[0, 1].set_title('NLTE')

    ax[2, 0].set_xlabel('Wavelength, nm')
    ax[2, 1].set_xlabel('Wavelength, nm')
    
    ax[0, 0].set_ylabel('ext / noext: hydro')
    ax[1, 0].set_ylabel('ext / noext: ssd')
    ax[2, 0].set_ylabel('ext / noext: 300G')

    for p, q in itertools.product(range(3), range(2)): 
    
        ax[p, q].set_ylim(0.8, 1.2)
        ax[p, q].set_xlim(180, 300)

#    leg = ax[0, 0].legend(framealpha = 1, loc = 2, handletextpad = 1, prop = {'size': 12.5})
    
    pdf_name = './ext_noext/mu_' + str(i) + '.pdf'
        
    plt.savefig(pdf_name, bbox_inches = 'tight')
    
    pdf_names += pdf_name + ' '
    
os.system('pdftk ' + pdf_names + ' output ./ext_noext/all.pdf')

os.system('rm ./ext_noext/mu_*.pdf')

100%|██████████| 5/5 [00:05<00:00,  1.17s/it]


0

In [16]:
I0l_m = np.load('spec_nessy_G2V_matthias_noext_32_mean.npz')['I0l']
ISl_m = np.load('spec_nessy_G2V_matthias_noext_32_mean.npz')['ISl']
I3l_m = np.load('spec_nessy_G2V_matthias_noext_32_mean.npz')['I3l']
I0n_m = np.load('spec_nessy_G2V_matthias_noext_32_mean.npz')['I0n']
ISn_m = np.load('spec_nessy_G2V_matthias_noext_32_mean.npz')['ISn']
I3n_m = np.load('spec_nessy_G2V_matthias_noext_32_mean.npz')['I3n']

In [17]:
Il_fal_q = np.zeros((9, 120))
Il_fal_f = np.zeros((9, 120))

In_fal_q = np.zeros((9, 120))
In_fal_f = np.zeros((9, 120))

Il_fal_q_tmin = np.zeros((9, 120))
Il_fal_f_tmin = np.zeros((9, 120))

In_fal_q_tmin = np.zeros((9, 120))
In_fal_f_tmin = np.zeros((9, 120))

Il_kur_q = np.zeros((9, 120))
Il_kur_f = np.zeros((9, 120))

In_kur_q = np.zeros((9, 120))
In_kur_f = np.zeros((9, 120))

idx_m = plt_mu
idx_n = [1, 3, 5, 7, 9]

for i in range(len(idx_m)):

    Il_fal_q[idx_m[i], :] = np.loadtxt('clv_lte_fal_q',  usecols = [idx_n[i]])
    Il_fal_f[idx_m[i], :] = np.loadtxt('clv_lte_fal_f',  usecols = [idx_n[i]])

    In_fal_q[idx_m[i], :] = np.loadtxt('clv_nlte_fal_q', usecols = [idx_n[i]])
    In_fal_f[idx_m[i], :] = np.loadtxt('clv_nlte_fal_f', usecols = [idx_n[i]])
    
    Il_fal_q_tmin[idx_m[i], :] = np.loadtxt('clv_lte_fal_q_tmin',  usecols = [idx_n[i]])
    Il_fal_f_tmin[idx_m[i], :] = np.loadtxt('clv_lte_fal_f_tmin',  usecols = [idx_n[i]])

    In_fal_q_tmin[idx_m[i], :] = np.loadtxt('clv_nlte_fal_q_tmin', usecols = [idx_n[i]])
    In_fal_f_tmin[idx_m[i], :] = np.loadtxt('clv_nlte_fal_f_tmin', usecols = [idx_n[i]])

    Il_kur_q[idx_m[i], :] = np.loadtxt('clv_lte_kur_q',  usecols = [idx_n[i]])
    Il_kur_f[idx_m[i], :] = np.loadtxt('clv_lte_kur_f',  usecols = [idx_n[i]])

    In_kur_q[idx_m[i], :] = np.loadtxt('clv_nlte_kur_q', usecols = [idx_n[i]])
    In_kur_f[idx_m[i], :] = np.loadtxt('clv_nlte_kur_f', usecols = [idx_n[i]])

In [18]:
pdf_names = ''

for i in tqdm(plt_mu):
    
    plt.close('all')

    fig, ax = plt.subplots(nrows = 3, ncols = 2, figsize = (15, 10))
    
    fig.suptitle(r'$\mu$ = ' + str(mu[i]), y = 1.01)

    fig.tight_layout()

    ax[0, 0].plot(w, I0l_m[i, :],         linewidth = 2, color = 'blue',    label = 'muram')
    
    ax[0, 0].plot(w, Il_fal_q_tmin[i, :], linewidth = 2, color = 'red',     label = 'fal w/o chromo')
    ax[0, 0].plot(w, Il_fal_q[i, :],      linewidth = 2, color = 'magenta', label = 'fal')
    
    ax[0, 0].plot(w, Il_kur_q[i, :],      linewidth = 2, color = 'cyan',    label = 'kurucz')
    
    #---------------------------------------------------------------------------------------------------

    ax[0, 1].plot(w, I0n_m[i, :],         linewidth = 2, color = 'blue',    label = 'muram')
    
    ax[0, 1].plot(w, In_fal_q_tmin[i, :], linewidth = 2, color = 'red',     label = 'fal w/o chromo')
    ax[0, 1].plot(w, In_fal_q[i, :],      linewidth = 2, color = 'magenta', label = 'fal')
    
    ax[0, 1].plot(w, In_kur_q[i, :],      linewidth = 2, color = 'cyan',    label = 'kurucz')
    
    #---------------------------------------------------------------------------------------------------

    ax[1, 0].plot(w, ISl_m[i, :],         linewidth = 2, color = 'blue',    label = 'muram')
    
    ax[1, 0].plot(w, Il_fal_q_tmin[i, :], linewidth = 2, color = 'red',     label = 'fal w/o chromo')
    ax[1, 0].plot(w, Il_fal_q[i, :],      linewidth = 2, color = 'magenta', label = 'fal')
    
    ax[1, 0].plot(w, Il_kur_q[i, :],      linewidth = 2, color = 'cyan',    label = 'kurucz')
    
    #---------------------------------------------------------------------------------------------------

    ax[1, 1].plot(w, ISn_m[i, :],         linewidth = 2, color = 'blue',    label = 'muram')
    
    ax[1, 1].plot(w, In_fal_q_tmin[i, :], linewidth = 2, color = 'red',     label = 'fal w/o chromo')
    ax[1, 1].plot(w, In_fal_q[i, :],      linewidth = 2, color = 'magenta', label = 'fal')
    
    ax[1, 1].plot(w, In_kur_q[i, :],      linewidth = 2, color = 'cyan',    label = 'kurucz')
    
    #---------------------------------------------------------------------------------------------------

    ax[2, 0].plot(w, I3l_m[i, :],         linewidth = 2, color = 'blue',    label = 'muram')
    
    ax[2, 0].plot(w, Il_fal_f_tmin[i, :], linewidth = 2, color = 'red',     label = 'fal w/o chromo')
    ax[2, 0].plot(w, Il_fal_f[i, :],      linewidth = 2, color = 'magenta', label = 'fal')
    
    ax[2, 0].plot(w, Il_kur_f[i, :],      linewidth = 2, color = 'cyan',    label = 'kurucz')
    
    #---------------------------------------------------------------------------------------------------

    ax[2, 1].plot(w, I3n_m[i, :],         linewidth = 2, color = 'blue',    label = 'muram')
    
    ax[2, 1].plot(w, In_fal_f_tmin[i, :], linewidth = 2, color = 'red',     label = 'fal w/o chromo')
    ax[2, 1].plot(w, In_fal_f[i, :],      linewidth = 2, color = 'magenta', label = 'fal')
    
    ax[2, 1].plot(w, In_kur_f[i, :],      linewidth = 2, color = 'cyan',    label = 'kurucz')

    ax[0, 0].set_title('LTE')
    ax[0, 1].set_title('NLTE')

    ax[0, 0].text(270, 5e-8, 'hydro')
    ax[0, 1].text(270, 5e-8, 'hydro')
    ax[1, 0].text(270, 5e-8, 'ssd')
    ax[1, 1].text(270, 5e-8, 'ssd')
    ax[2, 0].text(270, 5e-8, '300G')
    ax[2, 1].text(270, 5e-8, '300G')

    ax[2, 0].set_xlabel('Wavelength, nm')
    ax[2, 1].set_xlabel('Wavelength, nm')
    ax[0, 0].set_ylabel('Intensity, cgs')
    ax[1, 0].set_ylabel('Intensity, cgs')
    ax[2, 0].set_ylabel('Intensity, cgs')

    for p, q in itertools.product(range(3), range(2)): 
    
        ax[p, q].set_yscale('log')
        
        ax[p, q].set_ylim(1e-9, 1e-5)
        ax[p, q].set_xlim(180, 300)

    leg = ax[0, 0].legend(framealpha = 1, loc = 2, handletextpad = 1, prop = {'size': 12.5})
    
    pdf_name = './ext_noext/models_comp_' + str(i) + '.pdf'
        
    plt.savefig(pdf_name, bbox_inches = 'tight')
    
    pdf_names += pdf_name + ' '
    
os.system('pdftk ' + pdf_names + ' output ./ext_noext/models_comp.pdf')

os.system('rm ./ext_noext/models_comp_*.pdf')

100%|██████████| 5/5 [00:15<00:00,  2.93s/it]


0